In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics transformers datasets huggingface_hub -q

!pip install torch torchvision torchaudio -q
!pip install matplotlib tqdm seaborn -q


In [ ]:
import os
import yaml

# Define dataset paths (Google Drive)
data_yaml = {
    'train': "/kaggle/input/hand-dataset/hand_datasets/training_dataset/training_data",
    'val': "/kaggle/input/hand-dataset/hand_datasets/validation_dataset/validation_data",
    'test': "/kaggle/input/hand-dataset/hand_datasets/test_dataset/test_data",
    'nc': 1,  # Only one class: hand
    'names': ['hand']
}

# Save the YAML file
yaml_path = "/kaggle/working/data.yaml"
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

print(f"✅ data.yaml created at: {yaml_path}")


In [ ]:
from ultralytics import YOLO

# Load the pre-trained model
model = YOLO("ultralytics/yolov8n.pt")  # Choose YOLOv8n version (lightweight)


In [ ]:
# Train YOLOv8
results = model.train(
    data=yaml_path,      # Specify the data.yaml file
    epochs=50,           # Train for 50 epochs
    batch=16,            # Batch size
    imgsz=640,           # Training image size
    device="cuda",       # Use GPU for training (if available)
    cache="/kaggle/working/yolo_cache"  # Specify a writable cache directory
)

print("✅ YOLOv8 finished training!")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read training results
results_path = "/kaggle/working/runs/detect/train2/results.csv"
df = pd.read_csv(results_path)

# Plot Loss Curve
plt.figure(figsize=(10, 5))
plt.plot(df['epoch'], df['train/box_loss'], label='Box Loss')
plt.plot(df['epoch'], df['train/cls_loss'], label='Class Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Read training results
results_path = "/kaggle/working/runs/detect/train2/results.csv"
df = pd.read_csv(results_path)

# Plot Loss Curve
plt.figure(figsize=(10, 5))
plt.plot(df['epoch'], df['train/box_loss'], label='Box Loss')
plt.plot(df['epoch'], df['train/cls_loss'], label='Class Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()
plt.show()

# Plot mAP (mean Average Precision) Curve
plt.figure(figsize=(10, 5))
plt.plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@50')
plt.plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@50-95')
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.title('Mean Average Precision (mAP) Curve')
plt.legend()
plt.show()

# Plot Precision & Recall Curve
plt.figure(figsize=(10, 5))
plt.plot(df['epoch'], df['metrics/precision(B)'], label='Precision')
plt.plot(df['epoch'], df['metrics/recall(B)'], label='Recall')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Precision & Recall Curve')
plt.legend()
plt.show()


In [ ]:
# Perform evaluation
metrics = model.val(data=yaml_path, split="test")

# Correct way to retrieve mAP values
print(f"✅ Test data mAP@50: {metrics.box.map50:.3f}")
print(f"✅ Test data mAP@50-95: {metrics.box.map:.3f}")


In [ ]:
import glob
import os
import cv2
from IPython.display import display
from PIL import Image as PILImage  # Use PIL to display images

# Select one test image
test_images = glob.glob("/kaggle/input/hand-dataset/hand_datasets/training_dataset/training_data/images/Inria_100.jpg")
sample_image = test_images[0]

# Perform prediction
results = model.predict(source=sample_image, save=True)

# ✅ Get the YOLOv8 prediction save directory
save_dir = results[0].save_dir  # Access results[0] to get the save directory
print(f"✅ YOLOv8 predictions are saved in: {save_dir}")

# ✅ Retrieve the saved prediction image path
saved_images = sorted(glob.glob(os.path.join(save_dir, "*.jpg")))  # Get all predicted images
if saved_images:
    output_image_path = saved_images[0]  # Select the first detected image
    print(f"✅ Prediction image path: {output_image_path}")

    # ✅ Display the detection result using PIL
    display(PILImage.open(output_image_path))
else:
    print("❌ No predicted images found. Please check if YOLOv8 executed correctly!")


In [ ]:
# Copy the runs directory to /kaggle/outputs
!cp -r /kaggle/working/runs /kaggle/outputs/